In [1]:
import torch

### 1 初始化输入，假设输入是(4,3)

In [5]:
x = [
  [1, 0, 1, 0], # Input 1
  [0, 2, 0, 2], # Input 2
  [1, 1, 1, 1]  # Input 3
 ]
x = torch.tensor(x, dtype=torch.float32)
x

tensor([[1., 0., 1., 0.],
        [0., 2., 0., 2.],
        [1., 1., 1., 1.]])

### 2 初始化Q、K、V的权重矩阵<br/>
Note: 通常在神经网络的初始化过程中，这些参数都是比较小的，一般会在Gaussian, Xavier and Kaiming distributions随机采样完成。

In [30]:
input_size = 4
dim = 3 # 期望的q、k、v的向量大小
w_q = torch.normal(0, 1, size=(input_size, dim))
w_k = torch.normal(0, 1, size=(input_size, dim))
w_v = torch.normal(0, 1, size=(input_size, dim))
w_q, w_k, w_v

(tensor([[-0.6414, -0.1720,  1.9698],
         [-1.6207, -1.6754, -0.7933],
         [-0.0769, -1.8329,  2.3125],
         [ 1.3645,  0.1617, -1.0871]]),
 tensor([[ 1.0163,  1.0256, -0.2779],
         [ 0.9789,  0.8850,  1.6448],
         [-1.0321, -1.5823,  2.1647],
         [-0.5028,  0.0264, -0.8818]]),
 tensor([[-0.6609,  0.5672, -0.1946],
         [-0.9092, -0.9523, -3.8973],
         [-0.8648,  1.1575, -0.7758],
         [-0.1448, -0.4639,  0.7533]]))

### 3 计算q, k, v

In [31]:
q = x @ w_q
k = x @ w_k
v = x @ w_v
q, k, v

(tensor([[-0.7183, -2.0049,  4.2824],
         [-0.5125, -3.0275, -3.7607],
         [-0.9746, -3.5186,  2.4020]]),
 tensor([[-0.0159, -0.5567,  1.8868],
         [ 0.9521,  1.8227,  1.5261],
         [ 0.4602,  0.3546,  2.6499]]),
 tensor([[-1.5257,  1.7248, -0.9704],
         [-2.1080, -2.8323, -6.2879],
         [-2.5797,  0.3086, -4.1144]]))

### 4 计算attention_score
每个word自身相乘

In [32]:
attn_score = q @ k.T
attn_score

tensor([[  9.2075,   2.1972,  10.3061],
        [ -5.4021, -11.7452, -11.2747],
        [  6.5065,  -3.6754,   4.6688]])

### 5 sorfmax(attn_score)

In [37]:
attn_score_softmax = torch.nn.functional.softmax(attn_score, dim=-1) # 在最后一个维度上计算softmax
attn_score_softmax

tensor([[2.4994e-01, 2.2558e-04, 7.4983e-01],
        [9.9545e-01, 1.7507e-03, 2.8027e-03],
        [8.6265e-01, 3.2651e-05, 1.3732e-01]])

每一行中的一个：本行所代表的word对于输入的总word的权重，本例输入了3个word

### 6 给value乘上score
以第一行为例：第一个word对第一、第二、第三个word的attention为：2.4994e-01, 2.2558e-04, 7.4983e-01。<br/>
乘以value为：
```python
2.4994e-01 * [-1.5257,  1.7248, -0.9704] = [-0.38133346,  0.43109651, -0.24254178]
2.2558e-04 * [-2.1080, -2.8323, -6.2879] = [-0.00047552, -0.00063891, -0.00141842]
7.4983e-01 * [-2.2254e+00,  1.0077e-05, -5.6498e-01] = [-1.66867168e+00,  7.55603691e-06, -4.23638953e-01]
```

In [68]:
weighted_values = v[:,None] * attn_score_softmax.T[:,:,None]
weighted_values

tensor([[[-3.8134e-01,  4.3110e-01, -2.4255e-01],
         [-1.5187e+00,  1.7169e+00, -9.6600e-01],
         [-1.3161e+00,  1.4879e+00, -8.3713e-01]],

        [[-4.7554e-04, -6.3893e-04, -1.4185e-03],
         [-3.6906e-03, -4.9586e-03, -1.1008e-02],
         [-6.8829e-05, -9.2478e-05, -2.0531e-04]],

        [[-1.9343e+00,  2.3141e-01, -3.0851e+00],
         [-7.2301e-03,  8.6496e-04, -1.1531e-02],
         [-3.5424e-01,  4.2379e-02, -5.6498e-01]]])

### 7 求和
把每个第一行相加
```python
    [-3.8134e-01,  4.3110e-01, -2.4255e-01] 
+   [-4.7554e-04, -6.3893e-04, -1.4185e-03]
+   [-1.9343e+00,  2.3141e-01, -3.0851e+00]
=   [-2.3162,  0.6619, -3.3291]
```

In [70]:
output = weighted_values.sum(dim=0)
output

tensor([[-2.3162,  0.6619, -3.3291],
        [-1.5297,  1.7128, -0.9885],
        [-1.6704,  1.5302, -1.4023]])